# CNN for Image Classification

In [86]:
import cv2
import os
import numpy as np
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import merge
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from skimage.transform import rescale, resize
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [87]:
def disp(I):
    I = np.uint8(I)
    cv2.imshow('image', I)
    cv2.waitKey(0)

In [88]:
melonoma_input_folder = '../Dataset/Images/melonoma-resized'
others_input_folder = '../Dataset/Images/others-resized'
ground_truth_input_folder = '../Dataset/Images/gt-resized'

## Process data 
* Read the images from the **(384, 384)** resized directory to preserve the square shape.
* Multiply them with their respective ground truths.
* Resize it to **(64, 64)**.
* Save it in seperate directories.

In [89]:
X_train = []
Y_train = []

In [90]:
iterator = 0
num_melonoma = len(os.listdir(melonoma_input_folder))
for file_name in os.listdir(melonoma_input_folder):
    train_img = cv2.imread(os.path.join(melonoma_input_folder, file_name))
    ground_truth = cv2.imread(os.path.join(ground_truth_input_folder, file_name[:12] + '_segmentation.png'))
    ground_truth[ground_truth >= 127] = 255
    ground_truth[ground_truth < 127] = 0
    ground_truth = ground_truth / 255
    if iterator < 0.9 * num_melonoma:
        cv2.imwrite(os.path.join('../Dataset/Classification_Data/Training/class_1_melonoma', file_name), cv2.resize(train_img * ground_truth, (64, 64)))
    else:
        cv2.imwrite(os.path.join('../Dataset/Classification_Data/Testing/class_1_melonoma', file_name), cv2.resize(train_img * ground_truth, (64, 64)))
    iterator = iterator + 1

In [91]:
iterator = 0
num_others = len(os.listdir(others_input_folder))
for file_name in os.listdir(others_input_folder):
    train_img = cv2.imread(os.path.join(others_input_folder, file_name))
    ground_truth = cv2.imread(os.path.join(ground_truth_input_folder, file_name[:12] + '_segmentation.png'))
    ground_truth[ground_truth >= 127] = 255
    ground_truth[ground_truth < 127] = 0
    ground_truth = ground_truth / 255
    if iterator < 0.9 * num_others:
        cv2.imwrite(os.path.join('../Dataset/Classification_Data/Training/class_0_others', file_name), cv2.resize(train_img * ground_truth, (64, 64)))
    else:
        cv2.imwrite(os.path.join('../Dataset/Classification_Data/Testing/class_0_others', file_name), cv2.resize(train_img * ground_truth, (64, 64)))
    iterator = iterator + 1

In [92]:
datagen = ImageDataGenerator(rescale = 1.0 / 255)

## CNN starts here

In [93]:
CNN_input_size = [64, 64, 3]
trained_classifier_path = '../Trained/classifier.h5'

## f1-loss
* Tried this loss
* Inferior results

In [94]:
def f1_loss(y_true, y_pred):
    TP = K.sum(y_true * y_pred)
    soft_precision_positive = TP / (K.sum(y_pred) + 1e-7)
    soft_recall_positive = TP / (K.sum(y_true) + 1e-7)
    f1_loss_positive = 2 * soft_precision_positive * soft_recall_positive / (soft_precision_positive + soft_recall_positive + 1e-7)
    TN = K.sum((1 - y_true) * (1 - y_pred))
    soft_precision_negative = TN / (K.sum(1 - y_pred) + 1e-7)
    soft_recall_negative = TN / (K.sum(1 - y_true) + 1e-7)
    f1_loss_negative = 2 * soft_precision_negative * soft_recall_negative / (soft_precision_negative + soft_recall_negative + 1e-7)
    return 1 - (0.99 * f1_loss_positive + 0.01 * f1_loss_negative)

## Weighted Binary Cross-Entropy
* Tried this loss
* Inferior results

In [95]:
def weighted_binary_crossentropy(y_true, y_pred):
    return -K.sum(0.99 * y_true * K.log(y_pred + 1e-7) + 0.01 * (1 - y_true) * K.log(1 - y_pred + 1e-7))

## CNN Architecture
* Tried VGG-19 architecture
* Always predicts 0 resulting in 0 f1-score
* The below simple architecture gives decent results

In [108]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv1'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv2'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv3'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv4'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Flatten())

model.add(Dense(128, activation = 'relu', name = 'dense_1'))
model.add(Dense(1, name = 'dense_2'))
model.add(Dropout(0.5))
model.add(Activation('sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 8, 32)          9248      
__________

## Training the model

In [109]:
callbacks = [ModelCheckpoint(trained_classifier_path, monitor = 'val_loss', save_best_only = True, verbose = 2)]

In [110]:
model.fit_generator(datagen.flow_from_directory('../Dataset/Classification_Data/Training/', batch_size = 16, 
                    class_mode = 'binary', target_size = (64, 64)), steps_per_epoch = 1800 / 4, epochs = 25, 
                    verbose = 1, validation_data = datagen.flow_from_directory('../Dataset/Classification_Data/Testing/', 
                    batch_size = 4, class_mode = 'binary',  target_size = (64, 64)), callbacks = callbacks, class_weight = {0:1, 1:8})

Found 1801 images belonging to 2 classes.
Found 199 images belonging to 2 classes.
Epoch 1/25
450/450 [==============================] - 61s 136ms/step - loss: 0.5819 - acc: 0.8122 - val_loss: 0.5441 - val_acc: 0.8141

Epoch 00001: val_loss improved from inf to 0.54412, saving model to ../Trained/classifier.h5
Epoch 2/25
450/450 [==============================] - 63s 139ms/step - loss: 0.5628 - acc: 0.8134 - val_loss: 0.5804 - val_acc: 0.8141

Epoch 00002: val_loss did not improve
Epoch 3/25
450/450 [==============================] - 63s 139ms/step - loss: 0.5546 - acc: 0.8159 - val_loss: 0.5282 - val_acc: 0.7889

Epoch 00003: val_loss improved from 0.54412 to 0.52824, saving model to ../Trained/classifier.h5
Epoch 4/25
450/450 [==============================] - 71s 157ms/step - loss: 0.5399 - acc: 0.8185 - val_loss: 0.5156 - val_acc: 0.7990

Epoch 00004: val_loss improved from 0.52824 to 0.51557, saving model to ../Trained/classifier.h5
Epoch 5/25
450/450 [============================

KeyboardInterrupt: 

## Testing

In [111]:
test_set = datagen.flow_from_directory('../Dataset/Classification_Data/Testing/', target_size = (64, 64), class_mode = 'binary')

Found 199 images belonging to 2 classes.


In [112]:
results = model.predict_generator(test_set)
y_true = test_set.classes
y_pred = np.rint(results)
cm = confusion_matrix(y_true,y_pred)

# Class - Melanoma
precision = cm[1][1] / (cm[1][1] + cm[0][1])
recall =  cm[1][1] / (cm[1][1] + cm[1][0])

f1 = 2 * precision * recall / (precision + recall)
print("F1 score for melanoma:", f1)

# Class - Others
precision = cm[0][0] / (cm[0][0] + cm[0][1])
recall = cm[0][0] / (cm[0][0] + cm[1][0])

f1 = 2 * precision * recall / (precision + recall)
print("F1 score for non-melanoma:", f1)

F1 score for melanoma: 0.338028169014
F1 score for non-melanoma: 0.85626911315


## f1-score for Melonoma : 0.338  
## f1-score for Non-melonoma : 0.856